In [1]:
# Imports
import numpy as np
import keras
from keras.datasets import imdb
import pandas as pd
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization, regularizers
from keras import optimizers
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
import tensorflow as tf
import random

np.random.seed(42)
# Print the number of GPUs
print("Num GPUs Available: ",
      len(tf.config.experimental.list_physical_devices('GPU')))


class soc_nn:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def run(self):
        # Building the model
        model = Sequential()
        model.add(Dense(18, input_dim=len(self.x_train[0])))
        # (Dense(50, activation='relu', input_shape=self.x_train[0].shape))
        # model.add(BatchNormalization())
        model.add(Dense(12, activation='relu'))
        model.add(Dense(120, activation='relu'))
        model.add(Dense(20, activation='relu'))
        # model.add(Dropout(.1))
        # model.add(BatchNormalization())
        model.add(Dense(1, activation='sigmoid'))

        # Optimizer
        adam = optimizers.Adam(lr=0.0001)
        adadelta = optimizers.Adadelta(lr=1.0, rho=0.95)
        rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9)

        # Compiling the model
        model.compile(loss='mean_absolute_error', optimizer=adadelta,
                      metrics=['accuracy', 'mse'])

        # [['accuracy'], ['accuracy', 'mse']]
        model.summary()

        # Training the model
        model.fit(self.x_train, self.y_train, epochs=100, validation_data=(self.x_test, self.y_test),
                  batch_size=500, verbose=1)

        # Evaluating the model on the training and testing set
        score = model.evaluate(self.x_train, self.y_train)
        print("\n Training Accuracy:", score[1])
        score = model.evaluate(self.x_test, self.y_test)
        print("\n Testing Accuracy:", score[1])

        # Running and evaluating the model
        # hist = model.fit(self.x_train, self.y_train,
        #                  batch_size=3,
        #                  epochs=10,
        #                  validation_data=(self.x_test, self.y_test),
        #                  verbose=2)

        score = model.evaluate(self.x_test, self.y_test, verbose=0)
        print("Accuracy: ", score[1])
            
        return model


def scale_between_0_and_1(df, label):
   return (df[label] + abs(min(df[label]))) / (max(df[label]) + abs(min(df[label])))


# Load the dataset
data_1 = pd.read_csv('data/Augmented_data/battery_data_2_dV-C-roundV.csv')
print("battery_data_1 has {} data points with {} variables each."
      .format(*data_1.shape))
data_2 = pd.read_csv('data/Augmented_data/battery_data_2_dV-C.csv')
print("battery_data_2 has {} data points with {} variables each."
      .format(*data_2.shape))


# Prepare input parameters
y_1 = scale_between_0_and_1(data_1, 'SoC')
x_1 = data_1.drop(['sample_id', 'date', 'actual_time', 'mode', 'B_E', 'I_m',
                   'T_1', 'T_2', 'T_3', 'delta_time', 'runtime', 'SoC'], axis=1)

# x_1['T'] = round((data_1['T_1'] + data_1['T_2'] + data_1['T_3']) / 3, 1)
x_1['I_m'] = round(data_1['mode'] * data_1['B_E'] * data_1['I_m'], 3)
x_1['I'] = scale_between_0_and_1(x_1, 'I_m')
x_1['C'] = scale_between_0_and_1(data_1, 'C')
x_1['U_b'] = scale_between_0_and_1(data_1, 'U_b')
x_1['dV'] = scale_between_0_and_1(data_1, 'dV')
x_1['dV2'] = scale_between_0_and_1(data_1, 'dV2')
x_1['dV3'] = scale_between_0_and_1(data_1, 'dV3')

xt_1 = pd.DataFrame()
xt_1['ch1'] = x_1['U_b']
xt_1['ch2'] = x_1['dV']
xt_1['ch3'] = x_1['dV2']
xt_1['ch4'] = x_1['dV3']
xt_1['ch5'] = x_1['C']
xt_1['ch6'] = round(data_1['mode'] * data_1['B_E'] * data_1['I_m'], 3)
xt_1['idle1'] = x_1['U_b']
xt_1['idle2'] = x_1['dV']
xt_1['idle3'] = x_1['dV2']
xt_1['idle4'] = x_1['dV3']
xt_1['idle5'] = x_1['C']
xt_1['idle6'] = round(data_1['mode'] * data_1['B_E'] * data_1['I_m'], 3)
xt_1['dsch1'] = x_1['U_b']
xt_1['dsch2'] = x_1['dV']
xt_1['dsch3'] = x_1['dV2']
xt_1['dsch4'] = x_1['dV3']
xt_1['dsch5'] = x_1['C']
xt_1['dsch6'] = round(data_1['mode'] * data_1['B_E'] * data_1['I_m'], 3)

xt_1['ch1'].values[x_1['I_m'] <= 0] = 0
xt_1['ch2'].values[x_1['I_m'] <= 0] = 0
xt_1['ch3'].values[x_1['I_m'] <= 0] = 0
xt_1['ch4'].values[x_1['I_m'] <= 0] = 0
xt_1['ch5'].values[x_1['I_m'] <= 0] = 0
xt_1['ch6'].values[x_1['I_m'] <= 0] = 0
xt_1['idle1'].values[x_1['I_m'] != 0] = 0
xt_1['idle2'].values[x_1['I_m'] != 0] = 0
xt_1['idle3'].values[x_1['I_m'] != 0] = 0
xt_1['idle4'].values[x_1['I_m'] != 0] = 0
xt_1['idle5'].values[x_1['I_m'] != 0] = 0
xt_1['idle6'].values[x_1['I_m'] != 0] = 0
xt_1['dsch1'].values[x_1['I_m'] >= 0] = 0
xt_1['dsch2'].values[x_1['I_m'] >= 0] = 0
xt_1['dsch3'].values[x_1['I_m'] >= 0] = 0
xt_1['dsch4'].values[x_1['I_m'] >= 0] = 0
xt_1['dsch5'].values[x_1['I_m'] >= 0] = 0
xt_1['dsch6'].values[x_1['I_m'] >= 0] = 0


xt_1['ch6'] = xt_1['ch6'] / (max(xt_1['ch6']))
xt_1['dsch6'] = xt_1['dsch6'] / abs(min(xt_1['dsch6']))



print(xt_1, y_1)
x_1 = xt_1


# Create training and testing dataset
x_1, y_1 = x_1.to_numpy(), y_1.to_numpy()
train_size = int(len(x_1)/2)
test_size = len(x_1) - train_size

x_train = x_1[:train_size]
y_train = y_1[:train_size]
x_test = x_1[train_size:]
y_test = y_1[train_size:]

print("x_train has {} data points with {} variables each.".format(*x_train.shape))
print("x_test has {} data points with {} variables each.".format(*x_test.shape))

soc_nn = soc_nn(x_train, y_train, x_test, y_test)
model = soc_nn.run()

Using TensorFlow backend.


Num GPUs Available:  1
battery_data_1 has 106710 data points with 17 variables each.
battery_data_2 has 106710 data points with 17 variables each.
            ch1       ch2      ch3       ch4       ch5       ch6     idle1  \
0       0.00000  0.000000  0.00000  0.000000  0.000000  0.000000  0.851419   
1       0.00000  0.000000  0.00000  0.000000  0.000000  0.000000  0.909850   
2       0.90985  0.285714  0.00000  0.000000  0.000000  0.277269  0.000000   
3       0.90985  0.285714  0.56026  0.674521  0.000055  0.277269  0.000000   
4       0.90985  0.285714  0.56026  0.674521  0.000115  0.277269  0.000000   
...         ...       ...      ...       ...       ...       ...       ...   
106705  0.00000  0.000000  0.00000  0.000000  0.000000  0.000000  0.869783   
106706  0.00000  0.000000  0.00000  0.000000  0.000000  0.000000  0.869783   
106707  0.00000  0.000000  0.00000  0.000000  0.000000  0.000000  0.869783   
106708  0.00000  0.000000  0.00000  0.000000  0.000000  0.000000  0.86978

In [3]:
pred = model.predict(x_test, batch_size=32, verbose=0)
for i in pred:
    print(i[0])

import csv
with open('asd.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(pred)

0.6141232
0.6140794
0.6140419
0.61400366
0.6139662
0.6146529
0.611625
0.6136255
0.6165382
0.61377716
0.61373335
0.6136959
0.6136591
0.61362225
0.61358404
0.6135465
0.61350894
0.6134714
0.6134332
0.61339426
0.6133553
0.61331785
0.613281
0.61324203
0.6132031
0.6131655
0.6131289
0.61309195
0.6130549
0.6133805
0.6118487
0.61316097
0.61312234
0.61275315
0.6141888
0.61314833
0.61162347
0.612933
0.6128973
0.6128596
0.6128225
0.61278063
0.6127422
0.6127051
0.61266875
0.6126324
0.61331767
0.6102881
0.61295897
0.61226505
0.615204
0.6124038
0.6123681
0.612002
0.61342514
0.6120382
0.61200106
0.611964
0.61192626
0.6118884
0.61185133
0.6118129
0.6117737
0.61173666
0.61170024
0.6116631
0.6116267
0.611591
0.6115539
0.6115161
0.61147827
0.6114398
0.61139995
0.6116949
0.61018735
0.61149853
0.6111307
0.61255467
0.6118808
0.6088549
0.61119974
0.6126551
0.6112298
0.61119264
0.61082613
0.6122479
0.610859
0.61082256
0.61078405
0.6107434
0.6107022
0.61066574
0.6106293
0.6105921
0.61055434
0.6105172
0.61048007